In [1]:
import cv2
import numpy as np

left_image = cv2.imread('D:/test/ML/Stereo_Matching/left.png',0).astype(np.float32)
right_image = cv2.imread('D:/test/ML/Stereo_Matching/right.png',0).astype(np.float32)

height, width = left_image.shape

In [2]:
D = 16
# for each pixel in left image
disparity_map = np.zeros((height, width), dtype = np.uint8)
for i in range(height):
    for  j in range(width):
        min_cost = 256
        disparity = 0
        # get D pixel from right image
        for d in range(1, D):
            cost = 255 if (j-d) < 0 else abs(left_image[i,j] - right_image[i, j-d])
            if cost < min_cost:
                min_cost = cost
                disparity = d
        disparity_map[i,j] = disparity*15
cv2.imwrite('D:/test/ML/Stereo_Matching/disparity_map3.png', disparity_map)


True

Using Disparity Space

In [3]:
import cv2
import numpy as np



def Disparity_Space(disparity_range):
     left_image = cv2.imread('D:/test/ML/Stereo_Matching/left.png',0).astype(np.float32)
     right_image = cv2.imread('D:/test/ML/Stereo_Matching/right.png',0).astype(np.float32)
     height, width = left_image.shape
     print(height, width) 
     Disparity_map = np.zeros((height, width, disparity_range), dtype = np.float32)
     for i in range(height):
          for j in range(width):
               for d in range(disparity_range):
                    Disparity_map[i,j,d] = 255 if (j-d) < 0 else abs(left_image[i,j] - right_image[i, j-d])
     depth_map = np.zeros((height,width), dtype = np.uint8)
     for i in range(height):
          for j in range(width):
               disparity = 0
               cost_min = 255
               for d in range(disparity_range):
                    if Disparity_map[i,j,d] < cost_min:
                         cost_min = Disparity_map[i,j,d]
                         disparity = d
               depth_map[i,j] = disparity * (255/disparity_range)
        
     cv2.imwrite('D:/test/ML/Stereo_Matching/disparity_map2.png', depth_map)

Disparity_Space(16)



288 384


Using Vectorization (AD)




In [4]:
def compute_cost(disparity_range):
    left_image = cv2.imread('D:/test/ML/Stereo_Matching/left.png',0).astype(np.float32)
    right_image = cv2.imread('D:/test/ML/Stereo_Matching/right.png',0).astype(np.float32)
    height, width = left_image.shape
    cost_space = np.zeros((height, width, disparity_range), dtype = np.float32)
    for i in range(disparity_range):
        left = left_image[:, d: width]
        right = right_image[:,0: width-d]
        cost_d = abs(left - right)
        cost_space[:, d:width, d] = cost_d
    scale = 255/ disparity_range
    depth = np.argmin(cost_space, axis = 2)
    depth = depth * scale
    depth = depth.astype(np.unit8)
    cv2.imwirte('D:/test/ML/Stereo_Matching/disparity_map3.png', depth)

Computer SAD

In [15]:
import cv2
import numpy as np
def computer_intergral(data):
    sum_axis0 = np.cumsum(data, axis = 0)
    sum_axis1 = np.cumsum(sum_axis0, axis = 1)
    return sum_axis1

def computer_cost(left_image, right_image,disparity_range):

    left = cv2.imread(left_image,0).astype(np.float32)
    right = cv2.imread(right_image,0).astype(np.float32)
    
    height, width = left.shape

    cost_space = np.full((height, width, disparity_range), 255, np.float32)
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost_d  = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost_d
    

    intergral = np.full((height, width, disparity_range), 255, np.float32)
    for j in range(disparity_range):
        intergral[:,:,j] = computer_intergral(cost_space[:,:, j])
    
    side = 2
    cost_SAD_space = np.full((height, width,disparity_range), 255, np.float32)
    for i in range(side+1, height-side):
        for j in range(side+1, width-side):
            for d in range(disparity_range):
                cost_SAD_space[i+side, j+side, d] = intergral[i+side, j+side,d] - intergral[i+side, j-side-1, d] - intergral[i-side-1, j+side, d] + intergral[i-side-1,j-side-1,d]
    scale = 255/ disparity_range
    depth_map = np.argmin(cost_SAD_space, axis = 2)
    depth_map = depth_map * scale
    depth_map = depth_map.astype(np.uint8)
    cv2.imwrite('D:/test/ML/Stereo_Matching/disparity_map4.png', depth_map)
computer_cost('D:/test/ML/Stereo_Matching/left.png', 'D:/test/ML/Stereo_Matching/right.png', 16)


Computer SAD using Vectorization

In [1]:
import numpy as np
import cv2
def computer_intergral(data):
    sum_0 = np.cumsum(data, axis = 0)
    sum_1 = np.cumsum(sum_0, axis=1)
    return sum_1
def computer_cost(left_image, right_image, disparity_range):
    left = cv2.imread(left_image,0).astype(np.float32)
    right = cv2.imread(right_image,0).astype(np.float32)
    
    height, width = left.shape

    cost_space = np.full((height, width, disparity_range), 255, np.float32)
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost_d  = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost_d

    intergral = np.zeros((height, width,  disparity_range), dtype = np.float32)
    for i in range(disparity_range):
        intergral[:, :, i] = computer_intergral(cost_space[:,:, i])
    cost_SAD_space = np.zeros((height, width, disparity_range), dtype = np.float32)
    side = 2
    cost_SAD_space[:height-side, :width-side,:] = intergral[side: , side:, :]
    cost_SAD_space[side+1: , side+1:, :] = cost_SAD_space[side+1:, side+1:, :] + intergral[:height-side-1, :width-side-1,:]
    cost_SAD_space[side+1:, :width-side, :] = cost_SAD_space[side+1:, :width-side, :] - intergral[:height-side-1, side:, :]
    cost_SAD_space[:height-side, side+1:, :] = cost_SAD_space[:height-side, side+1:, :] - intergral[side:, :width-side-1, :]
    scale = 255 / disparity_range    
    depth = np.argmin(cost_SAD_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
    cv2.imwrite('D:/test/ML/Stereo_Matching/disparity_map5.png', depth)
computer_cost('D:/test/ML/Stereo_Matching/left.png', 'D:/test/ML/Stereo_Matching/right.png', 16)

Compute SAD ( using cosine similarity)

In [ ]:
import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def compute_integral_image(data):
    sum_axis_0 = np.cumsum(data, axis=0)
    sum_axis_1 = np.cumsum(sum_axis_0, axis=1)
    return sum_axis_1

def stereo_matching(left_img, right_img, disparity_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    left  = left.astype(np.float32)
    right = right.astype(np.float32)
    
    # compute integral image space
    integral_xy = np.full((height, width, disparity_range), 255*255, np.float32)
    integral_xx = np.full((height, width), 255*255, np.float32)
    integral_yy = np.full((height, width), 255*255, np.float32)    
   
    integral_xx = compute_integral_image(left*left)
    integral_yy = compute_integral_image(right*right)
    for j in range(disparity_range):        
        integral_xy[:, j:width, j] = left[:,j:width]*right[:, :width-j]
        integral_xy[:,:,j] = compute_integral_image(integral_xy[:,:,j])
        
    # compute sum
    side = 2
    integral_xy_sum = np.full((height, width, disparity_range), 255*255, np.float32)    
    integral_xy_sum[:height-side, :width-side, :] = integral_xy[side:, side:, :]
    integral_xy_sum[side+1:, side+1:, :] = integral_xy_sum[side+1:, side+1:, :] + integral_xy[:height-side-1, :width-side-1, :]
    integral_xy_sum[side+1:, :width-side, :] = integral_xy_sum[side+1:, :width-side, :] - integral_xy[:height-side-1, side:, :]
    integral_xy_sum[:height-side, side+1:, :] = integral_xy_sum[:height-side, side+1:, :] - integral_xy[side:, :width-side-1, :]
    
    integral_xx_sum = np.full((height, width), 255*255, np.float32)    
    integral_xx_sum[:height-side, :width-side] = integral_xx[side:, side:]
    integral_xx_sum[side+1:, side+1:] = integral_xx_sum[side+1:, side+1:] + integral_xx[:height-side-1, :width-side-1]
    integral_xx_sum[side+1:, :width-side] = integral_xx_sum[side+1:, :width-side] - integral_xx[:height-side-1, side:]
    integral_xx_sum[:height-side, side+1:] = integral_xx_sum[:height-side, side+1:] - integral_xx[side:, :width-side-1]
    
    integral_yy_sum = np.full((height, width), 255*255, np.float32)    
    integral_yy_sum[:height-side, :width-side] = integral_yy[side:, side:]
    integral_yy_sum[side+1:, side+1:] = integral_yy_sum[side+1:, side+1:] + integral_yy[:height-side-1, :width-side-1]
    integral_yy_sum[side+1:, :width-side] = integral_yy_sum[side+1:, :width-side] - integral_yy[:height-side-1, side:]
    integral_yy_sum[:height-side, side+1:] = integral_yy_sum[:height-side, side+1:] - integral_yy[side:, :width-side-1]
    
    # compute cost
    cost_space = np.full((height, width, disparity_range), -1.0, np.float32)
    for j in range(disparity_range):
        denor = np.sqrt(integral_xx_sum[:, j:width])*np.sqrt(integral_yy_sum[:, :width-j])
        cost_space[:, j:width, j] = integral_xy_sum[:, j:width, j] / denor
        
    # compute disparity from cost space
    scale = 255 / disparity_range    
    depth = np.argmax(cost_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_cosine.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)